In [113]:
# get scores from sagemaker aws private workforces

from boto3 import client
import boto3
import json
import pandas as pd

s3 = boto3.resource('s3')
col_names =  ['image', 'label']
my_df  = pd.DataFrame(columns = col_names)

conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3

for key in conn.list_objects(Bucket='workattr',Prefix='iteration-1')['Contents']:
    
    content_object = s3.Object('workattr', key['Key'])
    
    body=content_object.get()['Body'].read()
    
    my_json = body.decode('utf8').replace("'", '"')
    
    data = json.loads(my_json)
    
    for x in data:
        
        my_df.loc[len(my_df)] = [x['dataObject']['s3Uri'],x['annotations'][0]['annotationData']['content'][36]]

my_df.sort_values("image", inplace = True) 
    
my_df.to_excel("latino_male_21.xlsx") 



In [28]:
# get scores from sagemaker aws public workforces - AMT

from boto3 import client
import boto3
import json
import pandas as pd

s3 = boto3.resource('s3')
col_names =  ['image','rater','label']
my_df  = pd.DataFrame(columns = col_names)

conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3

for key in conn.list_objects(Bucket='workattr',Prefix='iteration-2')['Contents']:
    
    content_object = s3.Object('workattr', key['Key'])
    
    body=content_object.get()['Body'].read()
    
    my_json = body.decode('utf8').replace("'", '"')
    
    data = json.loads(my_json)
    
    for x in data:
         
        for y in x['annotations']:
            my_df.loc[len(my_df)] = [x['dataObject']['s3Uri'],y['workerId'],y['annotationData']['content']]

my_df.sort_values("image", inplace = True) 

#my_df['image'] = my_df['image'].str.replace(r'\D', '') 
my_df['label']= my_df['label'].str.replace(r'{"crowd-image-classifier":{"label":"', '') 
my_df['image']= my_df['image'].str.replace(r's3://ecust-fbp-male/', '')  
my_df['rater']= my_df['rater'].str.replace(r'public.us-east-1.', '')  
my_df['label']= my_df['label'].str.replace(r'"}}', '') 
my_df.to_excel("public_date_all5_2.xlsx")
#my_df

In [143]:
#create score matrix - private scores
import os
import pandas as pd
DIR ='/home/ec2-user/SageMaker/private_generic')
files = os.listdir(DIR)
files_xls = [f for f in files if f[-4:] == 'xlsx']

df=pd.DataFrame(columns = ['image','label'])
prefix_x=''

for f in files_xls:
    prefix_y=f
    data = pd.read_excel(f, 'Sheet1',usecols = "B:C")    
    df=pd.merge(df,data,on = "image", how='outer', suffixes=(prefix_x, prefix_y))
    prefix_x=f
    
#df['mean'] = df.mean(axis=1)    
df.to_excel("private_generic.xlsx")  

'\ndf = pd.read_excel("gen_guys1.xlsx")\ndf2 = pd.read_excel("gen_guys2.xlsx")\ndf3 = pd.read_excel("gen_guys3.xlsx")\ndf4 = pd.read_excel("gen_guys4.xlsx")\nmatrix = df.pivot_table(columns=\'rater\', index=\'image\', values=\'label\').reset_index()\nmatrix2 = df2.pivot_table(columns=\'rater\', index=\'image\', values=\'label\').reset_index()\nmatrix3 = df3.pivot_table(columns=\'rater\', index=\'image\', values=\'label\').reset_index()\nmatrix4 = df4.pivot_table(columns=\'rater\', index=\'image\', values=\'label\').reset_index()\n#matrix.columns.intersection(matrix2.columns)\nresult = pd.merge(matrix, matrix2, on="image")\nresult = pd.merge(result, matrix3, on="image")\nresult = pd.merge(result, matrix4, on="image")\n\n\nresult[\'mean\'] = result.mean(axis=1)\n\nresult.to_excel("guys_mean.xlsx")\n'

In [ ]:
#common image names
female = pd.read_excel('private_date_female.xlsx') 
male = pd.read_excel('private_date_male.xlsx')
s1 = pd.merge(female, male, how='inner', on=['image'])
s1

In [6]:
import zipfile
#os.chdir('/home/ec2-user/SageMaker')
with zipfile.ZipFile("images.zip","r") as zip_ref:
    zip_ref.extractall("")

In [15]:
import os
list = os.listdir("/home/ubuntu/mtcnn_aligned_images/female/caucasian/") # dir is your directory path
number_files = len(list)
number_files

542

In [ ]:
#images cropping
from PIL import Image
from autocrop import Cropper
import os
cropper = Cropper()
files=os.listdir('4-5')
for file in files:
    print(file)
    img_path = os.path.join('4-5', file)
    
    try:    
        cropped_array = cropper.crop(img_path)
        cropped_image = Image.fromarray(cropped_array)
        width, height = cropped_image.size
        print(width, height)
        cropped_image.save('./4_crop/'+file)
    except:
        print('no face',file)

In [15]:
import shutil
shutil.make_archive('scores', 'zip', './scores/')

'/home/ec2-user/SageMaker/scores.zip'

In [ ]:
#create score matrix - public AMT scores
import os
import pandas as pd
DIR = '/home/ec2-user/SageMaker/public_date/'
files = os.listdir(DIR)
files_xls = [f for f in files if f[-4:] == 'xlsx']
df=pd.DataFrame(columns = ['image','rater'])
prefix_x=''

for f in files_xls:
   # if '_male' in f:
       print(f) 
       data = pd.read_excel(DIR+f, 'Sheet1',usecols = "B:D") 
       data['image']= data['image'].str.replace(r's3://allattrimages/', '') 
       matrix = data.pivot_table(columns='rater', index='image', values='label').reset_index()
       df=pd.merge(df,matrix,on = "image", how='outer')

df1=df['image']
df = df.drop(['image', 'rater'], axis=1)
df3 = df.groupby(df.columns.str.split("_").str[0],axis=1).mean()
df3 = df3.join(df1) 
print(df3)
df3['mean'] = df3.mean(axis=1)
print(df3)
df3.to_excel("public_date.xlsx")

In [8]:
#join generic scores

import pandas as pd
private = pd.read_excel('') 
public = pd.read_excel('/home/ubuntu/ECUST_FBP/scores/public_generic_all.xlsx') 
private['image'] = private['image'].astype(str)+'.jpg'
all_=pd.merge(private,public,on = "image", how='outer')
all_.to_excel("/home/ubuntu/ECUST_FBP/scores/generic_all.xlsx")
#print(all_)

In [13]:
#add file path 
import os
import pandas as pd

df['path'] = ''      
        
        
        
df = pd.read_excel('/home/ubuntu/ECUST_FBP/scores/generic_all_deleted_small_rows_columns.xlsx') 
for index,row in df.iterrows():
        for root, dirs, files in os.walk('/home/ubuntu/ECUST_FBP/images/'):
            if row['image'] in files:
                df.loc[index,'path'] = os.path.join(root, row['image'])

            
#df['path'] = find(df['image'],'/home/ubuntu/ECUST_FBP/images/')
#df['path']
df.to_excel("/home/ubuntu/ECUST_FBP/scores/generic_all_deleted_small_rows_columns_path.xlsx")

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
df = pd.read_excel('/home/ubuntu/ECUST_FBP/scores/generic_all_deleted_small_rows_columns_path.xlsx')
df1 = df['image']
df2 = df['path']
df_t = df.T
df_t = df_t.drop(['image','path','Unnamed: 0'], axis = 0)
for column in df_t:

df=df_t.T
df=df.join(df1)
df=df.join(df2)
df.to_excel("/home/ubuntu/ECUST_FBP/scores/test.xlsx")